![A soccer pitch for an international match.](data/football-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in football analysis and reporting. You've been watching both men's and women's international football matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international football matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international football matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international football matches is greater than men's.

In [ ]:
import pandas as pd
import numpy as np

### Project Instructions

Perform an appropriate hypothesis test to determine the p-value, and hence result, of whether to reject or fail to reject the null hypothesis that the mean number of goals scored in women's international soccer matches is the same as men's. Use a 10% significance level.

For this analysis, you'll use Official FIFA World Cup matches since 2002-01-01, and you'll also assume that each match is fully independent, i.e., team form is ignored.

The p-value and the result of the test must be stored in a dictionary called `result_dict` in the form:

```python
result_dict = {"p_val": p_val, "result": result}
```
where `p_val` is the p-value and result is either the string "fail to reject" or "reject", depending on the result of the test.

In [7]:
url = "https://raw.githubusercontent.com/jhlopesalves/data-science-practice-notebook/refs/heads/main/Python/projects/hypothesis_test_football_matches/data/men_results.csv"
men_results = pd.read_csv(
    url,
    usecols=lambda col: not col.startswith("Unnamed"),
    parse_dates=["date"],
)
men_results.head()

,date,home_team,away_team,home_score,away_score,tournament
0,1872-11-30,Scotland,England,0,0,Friendly
1,1873-03-08,England,Scotland,4,2,Friendly
2,1874-03-07,Scotland,England,2,1,Friendly
3,1875-03-06,England,Scotland,2,2,Friendly
4,1876-03-04,Scotland,England,3,0,Friendly


In [8]:
url = "https://raw.githubusercontent.com/jhlopesalves/data-science-practice-notebook/refs/heads/main/Python/projects/hypothesis_test_football_matches/data/women_results.csv"
women_results = pd.read_csv(
    url,
    usecols=lambda col: not col.startswith("Unnamed"),
    parse_dates=["date"],
)
women_results.head()

,date,home_team,away_team,home_score,away_score,tournament
0,1969-11-01,Italy,France,1,0,Euro
1,1969-11-01,Denmark,England,4,3,Euro
2,1969-11-02,England,France,2,0,Euro
3,1969-11-02,Italy,Denmark,3,1,Euro
4,1975-08-25,Thailand,Australia,3,2,AFC Championship


In [ ]:
men_2002 = men_results[men_results["date"] >= "2002-01-01"]
women_2002 = women_results[women_results["date"] >= "2002-01-01"]

In [20]:
# avoid SettingWithCopyWarning by working on explicit copies
men_2002 = men_2002.copy()
women_2002 = women_2002.copy()

men_2002["total_score"] = men_2002["home_score"] + men_2002["away_score"]
women_2002["total_score"] = women_2002["home_score"] + women_2002["away_score"]

# restrict to official FIFA World Cup matches
men_wc = men_2002[men_2002["tournament"] == "FIFA World Cup"]
women_wc = women_2002[women_2002["tournament"] == "FIFA World Cup"]

# convenience series for the hypothesis test
men_scores = men_wc["total_score"]
women_scores = women_wc["total_score"]

men_scores.shape, women_scores.shape

((384,), (200,))

In [24]:
from scipy import stats

# compute descriptive stats
men_mean = men_scores.mean()
men_std = men_scores.std(ddof=1)

women_mean = women_scores.mean()
women_std = women_scores.std(ddof=1)

# two-sample Welch t-test (two-sided): H0 = no difference in means
t_stat, p_val = stats.ttest_ind(
    women_scores, men_scores, equal_var=False, nan_policy="omit"
)

alpha = 0.1
result = "reject" if p_val < alpha else "fail to reject"

result_dict = {"p_val": p_val, "result": result}
display(result_dict)

{'p_val': np.float64(0.0051961448009743005), 'result': 'reject'}